In [2]:
using DistMesh, Plots

In [36]:
fd1(p) = sqrt.(sum(p.^2)) - 1;
fh1(p) = ones(size(p,1),1);
(p, t) = distmesh( fd, fh, 0.2, [-1 -1;1 1] );
#plotgrid( p, t )

LoadError: MethodError: no method matching distmesh(::var"#1#2", ::var"#3#4", ::Float64, ::Matrix{Int64})
[0mClosest candidates are:
[0m  distmesh(::Function, ::Union{HUniform, Function}, ::Number) at C:\Users\sebas\.julia\packages\DistMesh\qIIr4\src\distmeshnd.jl:18
[0m  distmesh(::Function, ::Union{HUniform, Function}, ::Number, [91m::DistMesh.AbstractDistMeshAlgorithm[39m; origin, widths, fix, stats) at C:\Users\sebas\.julia\packages\DistMesh\qIIr4\src\distmeshnd.jl:18
[0m  distmesh(::Function, ::Any, ::Number, [91m::DistMeshSetup[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Val{stats}[39m, [91m::Type{VertType}[39m) where {VertType, stats} at C:\Users\sebas\.julia\packages\DistMesh\qIIr4\src\distmeshnd.jl:50

In [35]:
x = 0:0.1:1
fd1(x)

0.9621416870348585

In [ ]:
function b = ensambleCarga2D(p,t,f)
  np = size(p,2);
  nt = size(t,2);
  b = zeros(np,1);
  for K = 1:nt
    loc2glb = t(1:3,K);
    x = p(1,loc2glb);
    y = p(2,loc2glb);
    area = polyarea(x,y);
    bK = [f(x(1),y(1)); f(x(2),y(2)); f(x(3),y(3))]/3*area; % Vector de Carga en el elemento
    b(loc2glb) = b(loc2glb) + bK; % adicions de las cargas por elemento a b
end

In [ ]:
function M = ensambleMasa2D(p,t)
  np = size(p,2); % numero de nodos
  nt = size(t,2); % numero de elementos
  M = sparse(np,np); %  inicializacion de la matriz de masa
for K = 1:nt %  bucle sobre los elementos
    loc2glb = t(1:3,K); % transformacion de local-a-global
    x = p(1,loc2glb); % coordenadas x del nodo
    y = p(2,loc2glb); % coordenadas y del nodo
    area = polyarea(x,y); % area del triangulo
    MK = [2 1 1; 1 2 1; 1 1 2]/12*area; %  Matriz de masa en el elemento
    M(loc2glb,loc2glb) = M(loc2glb,loc2glb) + MK; % Sumando elementos a la matriz M
end

In [ ]:
function L2Proyeccion2D()
% Si usa MATLAB con el PDEtool puede utilizar las sgte instruccions
%g = Rectg(0,0,1,1); % cuadrado unitario
%[p,e,t] = initmesh(g,'hmax',0.1); % cree la malla


% Usando Distmesh
fd=@(p) drectangle(p,0,1,0,1); % cuadrado unitario
[p,t]=distmesh2d(fd,@huniform,0.1,[0,0;1,1],[0,0;1,0;0,1;1,1]); % cree la malla
clc

f = @(x,y)  25*((x-0.5).^2 + (y-0.5).^2) ;
%range = linspace (0, 1, 41);
%[X, Y] = meshgrid (range, range);
%Z = f (X, Y);
%figure
%surf (X, Y, Z);
%

M = ensambleMasa2D(p',t'); %  ensamble matiz de masa
b = ensambleCarga2D(p',t',f); %Foo2); % ensamble vector de carga
Pf = M\b; % Solucion del sistema lineal
figure
show(t,p,full(Pf)); % Dibuje la proyeccion
xlabel('X')
ylabel('Y')
view(-30,30)

% pdesurf(p',t',Pf) % Dibuje la proyeccion usando Matlab con pdetool

In [ ]:
function show(T,P,U)
%  T  matriz de conectividad  elementos
%  P  matriz de coordenadas de los nodos
%  U  Función a graficar sobre la malla de elemetos finitos

trisurf(T,P(:,1),P(:,2),U','facecolor','interp')
colormap jet
view(0,90)
%view(10,40);
title('Proyección en L^2')